In [1]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('stb_scratch').setMaster('local[*]')
sc = SparkContext(conf=conf)

lines = sc.textFile("/Users/snehalbhaitandel/Projects/data/Set_Top_Box_Data.txt")
d_id = lines.map(lambda x:x.split("^")[5])
xmld = lines.map(lambda x:x.split("^")[4])

d_id = lines.map(lambda x:x.split("^")[5])

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col
sqlc = SQLContext(sc)
rdd = lines.map(lambda x:x.split("^"))
schema = StructType([StructField("Server_Unique_ID", StringType(), True), StructField("Test_ID", StringType(), False),\
                                 StructField("Event_ID", StringType(), False), StructField("Time", StringType(), False), \
                                 StructField("XMLD", StringType(), False), StructField("Device_ID", StringType(), False), \
                                StructField("Secondary_Time", StringType(), False)])
df = sqlc.createDataFrame(rdd, schema=schema)


In [18]:
from pyspark.sql.functions import udf
import datetime
import xml.etree.ElementTree as ET
import pyspark.sql.functions as F
def xml_data(x):
    raw = ET.fromstring(x)
    for nv in raw.iter("nv"):
        kv = dict(nv.attrib)
        k = kv["n"]
        v = kv["v"]
        if k == "Duration":
            return v

xml_udf = udf(lambda x: xml_data(x))

df = df.withColumn("Duration", xml_udf("XMLD").cast(LongType()))
# df.select("Duration").count()
# df.write.format("csv").option("header", "true").save("/Users/snehalbhaitandel/Projects/data/stb"+str(datetime.datetime.now()))

In [20]:
df.show()

+----------------+-------+--------+--------------------+--------------------+--------------------+--------------+--------+
|Server_Unique_ID|Test_ID|Event_ID|                Time|                XMLD|           Device_ID|Secondary_Time|Duration|
+----------------+-------+--------+--------------------+--------------------+--------------------+--------------+--------+
|           11001|      1|     100|2015-06-05 22:35:...|<d><nv n="ExtStat...|0122648d-4352-4ee...|    2016060601|   24375|
|           11001|      1|     100|2015-06-05 22:35:...|<d><nv n="ExtStat...|0122648d-4352-4ee...|    2016060601| 1520565|
|           11001|      1|     100|2015-06-05 23:01:...|<d><nv n="ExtStat...|0122648d-4352-4ee...|    2016060601| 2926708|
|           11001|      1|     100|2015-06-06 00:29:...|<d><nv n="ExtStat...|0122648d-4352-4ee...|    2016060601|  924979|
|           11001|      1|     100|2015-06-06 00:44:...|<d><nv n="ExtStat...|0122648d-4352-4ee...|    2016060601|  716513|
|           1100

xmld = lines.map(lambda x:x.split("^")[4])